In [1]:
%load_ext jupyter_black

In [2]:
import re
import json
import time
import requests
from datetime import datetime, date

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import tabula
import pandas as pd

## The Govtrack ideology scores downloaded from Govtrack.us
The 2020 report dates from Jan 3 -2019 to Jan 3 - 2021 whereas the 2022 report is dated Jan 3 - 2021 to Jan 3 - 2022. The Govtrack 
report of the current legislators is yet to be released.

In [3]:
# read the 2020 reports and merge
df_2020_rep = pd.read_csv("raw_data/govtrack-stats-2020-house-ideology.csv")
df_2020_sen = pd.read_csv("raw_data/govtrack-stats-2020-senate-ideology.csv")

df_2020 = pd.concat([df_2020_rep, df_2020_sen])
df_2020.district = df_2020.district.fillna("Senator")
df_2020["period"] = 2020
df_2020.district = df_2020.district.apply(lambda dis: dis if type(dis) == str else dis)

# read the 2022 reports
df_2022_rep = pd.read_csv("raw_data/govtrack-stats-2022-house-ideology.csv")
df_2022_sen = pd.read_csv("raw_data/govtrack-stats-2022-senate-ideology.csv")

df_2022 = pd.concat([df_2022_rep, df_2022_sen])
df_2022["period"] = 2022
df_2022.district = df_2022.district.fillna("Senator")
df_2022.district = df_2022.district.apply(lambda dis: dis if type(dis) == str else dis)

com### Religious denominations in Congress

In [3]:
# read religion composition tables

dfs_religion_2020 = tabula.read_pdf(
    "raw_data/Faith-on-the-Hill-116-detailed-tables.pdf", pages="all"
)
dfs_religion_2022 = tabula.read_pdf(
    "raw_data/01.04.21_faith_on_the_hill_detailed.table_.update.pdf", pages="all"
)

In [4]:
def concat_pdf(dfs_, year):
    "Concatenate religious denominations"
    dfs = []
    for i, df in enumerate(dfs_):
        if i == 0 and year == 2020:
            pass
        else:
            df = pd.concat([df, df.columns.to_frame().T], ignore_index=True)

            if year == 2022:
                df.columns = [
                    "State",
                    "District",
                    "Name",
                    "Party",
                    "Freshman",
                    "Denominational family",
                ]
            else:
                df.columns = dfs_religion_2020[0].columns

        df.District = df.District.apply(
            lambda x: "Senator"
            if x == "Sen" or x == "Senator"
            else "0"
            if x == "At-Large" or x == "At-large"
            else x
        )

        dfs.append(df)

    return pd.concat(dfs)

In [5]:
df_religion_2020 = concat_pdf(dfs_religion_2020, 2020)
df_religion_2022 = concat_pdf(dfs_religion_2022, 2022)

# concatenate last and full name
df_religion_2020["Full_Name"] = df_religion_2020.apply(
    lambda row: row["First/middle"] + " " + row["Last"], axis=1
)

## Veterans in Congress - Data from (militarytimes.com)

In [6]:
def get_veterans(url):
    # using selenium to acquire veterans in the congress. Data source infogram

    # Create a new Chrome session
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 20)

    # Load the web page
    driver.get(url)  # Wait for the page to fully load
    driver.maximize_window()

    wait.until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, ".igc-table-container"))
    )
    # short pause added in order to make sure all the elements are loaded after we know the first element was loaded
    time.sleep(0.5)

    soup = BeautifulSoup(driver.page_source, "lxml")

    div = soup.select_one("div.igc-table-container")

    df = pd.read_html(str(div))[0]

    df["State"] = df.District.apply(
        lambda dis: dis.split(" ")[0] if dis != "Georgia" else "GA"
    )
    df["District"] = df.District.apply(
        lambda dis: "0"
        if len(dis.split(" ")) == 1 or dis.split(" ")[1] == "(House)"
        else str(dis.split(" ")[1])
    )
    df["District"] = df.District.apply(
        lambda dis: "Senator" if dis == "Sen" or dis == "Senate" else dis
    )
    df["Party"] = df.Party.apply(
        lambda party: "R"
        if party == "GOP" or party == "Republican"
        else "D"
        if party == "Dem"
        else party
    )

    return df

In [7]:
df_mil_2020 = get_veterans(
    "https://e.infogram.com/b28274cb-26b9-441f-9994-a8eb8558839b?src=embed"
)
df_mil_2020.to_csv("raw_data/veterans_in_congress_116.csv", index=False)

df_mil_2022 = get_veterans(
    "https://infogram.com/veterans-in-117th-congress-1hxj48ppv9zz52v"
)
df_mil_2022.to_csv("raw_data/veterans_in_congress_117.csv", index=False)

In [8]:
# Merge the Congressmen religion and "is veteran" dataframes

df_mil_2020

,District,Party,Name,In office now?,Gender,Service,Era,Combat tours,State
0,9,R,Doug Collins,Yes,Male,AF Reserve,2000s-2010s,OIF,GA
1,Senator,R,Mike Enzi,Yes,Male,Air Force,1960s-1970s,none,WY
2,Senator,R,Lindsey Graham,Yes,Male,Air Force,1980s-2010s,OIF,SC
3,6,D,Chrissy Houlahan,No,Female,Air Force,1980s,none,PA
4,5,R,Denver Riggleman,No,Male,Air Force,1990s,none,VA
...,...,...,...,...,...,...,...,...,...
91,1,R,Andy Harris,Yes,Male,Navy Reserve,1980s-2010s,Desert Storm,MD
92,20,D,Jimmy Panetta,Yes,Male,Navy Reserve,2000s,OEF,CA
93,3,R,Jim Banks,Yes,Male,Navy Reserve,2010s,OEF,IN
94,4,R,Steven Palazzo,Yes,Male,Marine Corps/Army,1980s-2010s,"Desert Storm, OIF",MS


In [9]:
# match the names and merge the dataframes using Levenshtein logic

import fuzzy_pandas as fpd

In [10]:
# rename religion dataframe

df_religion_2020.rename(
    columns={"Full_Name": "Name_Rel", "Last": "Last_Name"}, inplace=True
)
df_mil_2020.rename(columns={"Name": "Name_Mil"}, inplace=True)

matches = fpd.fuzzy_merge(
    df_religion_2020,
    df_mil_2020,
    left_on=["Name_Rel", "State", "District", "Party"],
    right_on=["Name_Mil", "State", "District", "Party"],
    method="levenshtein",
    threshold=0.55,
    ignore_case=True,
    ignore_nonalpha=True,
    ignore_nonlatin=True,
    ignore_order_words=True,
)
matches["Military"] = "Yes"

# left join df_religion with matches

df_religion_2020 = df_religion_2020.merge(
    matches[["Name_Rel", "Name_Mil", "Last_Name", "Military"]], how="left"
)
df_religion_2020.Military = df_religion_2020.Military.fillna("No")
df_religion_2020.Last_Name = df_religion_2020.Last_Name.apply(
    lambda name: name.split(" ")[0]
)

## Other Congress biographies and social links - source https://github.com/unitedstates/congress-legislators

In [11]:
congress_historical = pd.read_csv("raw_data/legislators-historical.csv")
congress_current = pd.read_csv("raw_data/legislators-current.csv")
congress_data = pd.concat([congress_historical, congress_current])
congress_data.reset_index(inplace=True, drop=True)

# congress 2020
congress_2020_senate = pd.read_csv(
    "raw_data/116th Congress Members Guide with Elections and Demographic Data by District - Senate.csv"
)
congress_2020_senate["Period"] = 116

congress_2020_house = pd.read_csv(
    "raw_data/116th Congress Members Guide with Elections and Demographic Data by District - House.csv"
)
congress_2020_house["Period"] = 116

# congress 2022
congress_2022_senate = pd.read_csv(
    "raw_data/117th Congress Members Guide with Election Results and Demographic Data by District - Senate.csv"
)
congress_2022_senate["Period"] = 117

congress_2022_house = pd.read_csv(
    "raw_data/117th Congress Members Guide with Election Results and Demographic Data by District - House.csv"
)
congress_2022_house["Period"] = 117


congress_senate = pd.concat([congress_2020_senate, congress_2022_senate])
congress_house = pd.concat([congress_2020_house, congress_2022_house])

In [12]:
congress_house["Distr"] = congress_house.Code.apply(
    lambda code: 0 if code.split("-")[1] == "AL" else code.split("-")[1]
)
congress_house["State"] = congress_house.Code.apply(
    lambda code: code.split("-")[0]
)

In [13]:
congress_senate.head()

,State,Year,Class,Party,First_Name,Last_Name,First Elected,Birth Year,Gender,Race_Ethnicity,Religion,LGBTQ,Pre_2018_Incumbent,Period,Appointed,Pre_2020 Incumbent
0,Alabama,2017,II,Democratic,Doug,Jones,2017 Special,1954,Man,White - Non-Hispanic,Christian - Methodist,No,Lost in primary,116,NaN,NaN
1,Alabama,2016,III,Republican,Richard,Shelby,1986,1934,Man,White - Non-Hispanic,Christian - Presbyterian,No,Not up,116,NaN,NaN
2,Alaska,2014,II,Republican,Dan,Sullivan,2014,1964,Man,White - Non-Hispanic,Christian - Roman Catholic,No,Not up,116,NaN,NaN
3,Alaska,2016,III,Republican,Lisa,Murkowski,2002 Appointment,1957,Woman,White - Non-Hispanic,Christian - Roman Catholic,No,Not up,116,NaN,NaN
4,Arizona,2018,I,Democratic,Kyrsten,Sinema,2018,1976,Woman,White - Non-Hispanic,Unaffiliated,Bisexual,Open - Retired,116,NaN,NaN


In [14]:
# A python dictionary to abbr US states

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [15]:
congress_senate.State = congress_senate.State.apply(
    lambda x: us_state_to_abbrev[x]
)

In [16]:
# concatenate the senate and the house dataframes
df_bios_2020 = pd.concat([congress_senate, congress_house])

df_bios_2020.drop(["Year", "Code", "District"], inplace=True, axis=1)
df_bios_2020.rename(
    columns={"Distr": "District", "Class": "Senate Class"}, inplace=True
)

df_bios_2020.District.fillna("Senator", inplace=True)
df_bios_2020["Type"] = df_bios_2020.District.apply(
    lambda district: "sen" if district == "Senator" else "rep"
)

df_bios_2020.reset_index(inplace=True, drop=True)
df_bios_2020.Gender = df_bios_2020.Gender.apply(
    lambda gender: "M" if gender == "Man" else "F" if gender == "Woman" else gender
)

In [17]:
congress_data.birthday = pd.to_datetime(congress_data.birthday)
congress_data["Birth Year"] = congress_data.birthday.dt.year

In [18]:
congress_data[congress_data.last_name.str.contains("Manchin")]

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,Birth Year
12162,Manchin,Joe,NaN,III,NaN,"Joe Manchin, III",1947-08-24,M,sen,WV,...,S338,S0WV00090,62864.0,412391,7547.0,Joe Manchin III,NaN,40915.0,Joe Manchin,1947.0


In [19]:
# Join the historical and bios data of the congress members

matches = df_bios_2020.merge(
    congress_data,
    right_on=["last_name", "state", "type", "gender", "Birth Year"],
    left_on=["Last_Name", "State", "Type", "Gender", "Birth Year"],
    how="left",
)

new_df = matches[~matches.govtrack_id.isna()]

In [20]:
# merge the new dataframe with the govtract scores using govtrack ids

df_ideology = new_df.merge(
    df_2020[["ideology", "id", "bioguide_id"]],
    left_on=["govtrack_id", "bioguide_id"],
    right_on=["id", "bioguide_id"],
    how="left",
)

In [21]:
df_ideology

,State,Senate Class,Party,First_Name,Last_Name,First Elected,Birth Year,Gender,Race_Ethnicity,Religion,...,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,ideology,id
0,AL,II,Democratic,Doug,Jones,2017 Special,1954,M,White - Non-Hispanic,Christian - Methodist,...,S0AL00156,1024592.0,412741.0,176464.0,Doug Jones (Alabama),NaN,41705.0,Doug Jones (politician),0.632886,412741.0
1,AL,III,Republican,Richard,Shelby,1986,1934,M,White - Non-Hispanic,Christian - Presbyterian,...,S6AL00013,1859.0,300089.0,53266.0,Richard Shelby,NaN,94659.0,Richard Shelby,0.610349,300089.0
2,AK,II,Republican,Dan,Sullivan,2014,1964,M,White - Non-Hispanic,Christian - Roman Catholic,...,S4AK00214,1023262.0,412665.0,114964.0,Daniel S. Sullivan,NaN,41500.0,Dan Sullivan (U.S. senator),0.714706,412665.0
3,AK,III,Republican,Lisa,Murkowski,2002 Appointment,1957,F,White - Non-Hispanic,Christian - Roman Catholic,...,S4AK00099,1004138.0,300075.0,15841.0,Lisa Murkowski,NaN,40300.0,Lisa Murkowski,0.568338,300075.0
4,AZ,I,Democratic,Kyrsten,Sinema,2018,1976,F,White - Non-Hispanic,Unaffiliated,...,"H2AZ09019,S8AZ00197",68489.0,412509.0,28338.0,Kyrsten Sinema,NaN,21300.0,Kyrsten Sinema,0.675805,412509.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,WI,NaN,Republican,Jim,Sensenbrenner,1978,1943.0,M,White - Non-Hispanic,Christian - Roman Catholic,...,H8WI09050,1507.0,400365.0,27142.0,Jim Sensenbrenner,NaN,14657.0,Jim Sensenbrenner,0.621411,400365.0
602,WI,NaN,Republican,Glenn,Grothman,2014,1955.0,M,White - Non-Hispanic,Christian - Lutheran,...,H4WI06048,77282.0,412661.0,3493.0,Glenn Grothman,NaN,21559.0,Glenn Grothman,0.816250,412661.0
603,WI,NaN,Republican,Tom,Tiffany,2020 Special,1957.0,M,White - Non-Hispanic,Christian,...,H0WI07101,NaN,456791.0,NaN,Tom Tiffany,NaN,NaN,Tom Tiffany,0.578802,456791.0
604,WI,NaN,Republican,Mike,Gallagher,2016,1984.0,M,White - Non-Hispanic,Christian - Roman Catholic,...,H6WI08155,104067.0,412731.0,171843.0,Michael Gallagher (Wisconsin),NaN,21720.0,Mike Gallagher (American politician),0.761699,412731.0


In [22]:
def age(born):
    try:
        today = date.today()
        return (
            today.year - born.year - ((today.month, today.day) < (born.month, born.day))
        )
    except (ValueError, TypeError):
        return born  # leave unchanged


df_ideology["age"] = df_ideology.birthday.apply(age)

In [23]:
df_ideology = df_ideology[~df_ideology.twitter.isna()]

In [24]:
# save the ideology dataframe

df_ideology.to_csv("raw_data/congress_bios.csv", index=False)

### Twitter data and merge with the Biography data

In [25]:
# read Twitter data

twitter_data = pd.read_csv("raw_data/vader_sentiment_tweets.csv")

In [26]:
twitter_data.head(5)

,tweet_id,username,party,tweet,clean_text,favorite_count,retweet_count,created_at,source,social_policy,...,rank,score,govtrack_cluster,progressive_cluster,govtrack_class,positive_sentiment,neutral_sentiment,negative_sentiment,compound_sentiment,sentiment_text
0,1453097115664662536,RepMikeGarcia,R,I joined fellow China Task Force lawmakers in ...,joined fellow china task force lawmakers in s...,43,29,2021-10-26 20:32:04+00:00,Twitter Web App,NaN,...,#218,0.44,2,3,Centrist,0.000001,0.807001,0.193001,-0.771699,negative
1,1511713375218245640,RepGallagher,R,"Rep. Gallagher: ""If we do not invest in hard p...",rep gallagher if we do not invest in hard powe...,25,11,2022-04-06 14:32:09+00:00,Twitter Web App,NaN,...,#52,0.76,4,4,Far Right,0.113001,0.786001,0.101001,-0.128999,negative
2,1381021863313350658,RepDLesko,R,"CBP is currently encountering over 5,000 peopl...",cbp is currently encountering over 000 people ...,283,82,2021-04-10 23:10:45+00:00,Twitter for iPhone,Social Policies,...,#7,0.91,4,4,Far Right,0.029001,0.860001,0.110001,-0.678299,negative
3,1362468292569534465,RepDebDingell,D,It’s past time our country ban discrimination ...,it is past time our country ban discrimination...,92,21,2021-02-18 18:25:29+00:00,Twitter Web App,Social Policies,...,#351,0.25,1,0,Left Centrist,0.086001,0.799001,0.115001,-0.226299,negative
4,1551368623142363138,RepCarbajal,D,Proud to present the Woman of the Year Award t...,proud to present the woman of the year award t...,17,1,2022-07-25 00:47:57+00:00,Twitter for iPhone,Social Policies,...,#283,0.32,1,0,Left Centrist,0.329001,0.671001,0.000001,0.940301,positive


In [27]:
# merge the the biographies with the twitter dataset on username

df = twitter_data.merge(df_ideology, left_on="username", right_on="twitter", how="left")

df.Party = df.Party.apply(
    lambda party: "R"
    if party == "Republican"
    else "D"
    if party == "Democratic"
    else "I"
    if party == "Independent"
    else party
)

In [28]:
df = df[~df.govtrack_id.isna()]

In [29]:
# include the military and religion

# df = df.merge(df_religion_2020[["Last_Name", "District", "State", "Party", "Military"]], on=["State", "Last_Name", "District", "Party"], how="left")
# df.Military = df["Military"].fillna("No")